In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/ECE324/ECE324 Project 
# %cd /content/drive/MyDrive/ECE324 Project

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1n2vGsCOPhE_6QOOxyZh4O8W37BWTgMRY/ECE324 Project


In [2]:
import json
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier

import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [3]:
# df = pd.read_csv("cleanedData.csv", sep='\t')
df = pd.read_csv("cleanedData_cuisinevec.csv", sep=',')
# df = df.rename(index={0: "stars", 1: "delivery", 2: "takeout", 3: "outdoor seating", 4: "closingTime", 5: "cost", 6: "cuisine", 7: "location"})
df = df.replace(np.nan,0)
df

,stars,delivery,takeout,Outdoor seating,hours,cost,location,cuisine1,cuisine2,cuisine3,...,cuisine41,cuisine42,cuisine43,cuisine44,cuisine45,cuisine46,cuisine47,cuisine48,cuisine49,cuisine50
0,4.0,1,1,1,15.0,2,0,0.461130,1.158050,-1.254000,...,0.373500,0.391910,-1.187350,0.462550,0.173133,0.889410,-0.447190,-0.074539,0.020047,0.872685
1,4.5,1,1,1,22.0,2,0,-0.239570,-1.112700,-0.451410,...,-0.614770,0.651480,0.374190,-0.389130,1.165600,-0.143990,-0.947270,0.185640,1.439500,-1.022900
2,4.5,1,1,0,21.0,2,0,-0.560245,0.416880,-0.129945,...,0.014230,0.155507,0.605303,-0.594330,-0.091074,0.521544,-0.575590,0.147694,-0.081906,0.242015
3,4.5,0,0,0,16.0,2,0,-0.085708,-0.044853,-0.929813,...,0.207813,0.555055,-0.048445,-0.052447,0.703163,0.650447,-0.613009,0.111827,0.255091,-0.653027
4,4.0,0,0,1,14.5,2,0,0.360702,0.393248,-1.283488,...,0.288502,0.437224,-0.188186,0.331956,0.499129,0.451081,-0.378374,-0.354534,0.220454,0.140336
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1025,3.5,1,1,1,22.0,2,2,0.156797,-0.716633,-0.494297,...,0.126953,0.564053,0.054940,0.352709,0.847907,0.835447,-0.113245,-0.099306,0.126680,0.651583
1026,4.0,0,0,0,21.5,2,2,-0.502720,-0.862780,-0.627790,...,-0.906370,0.861670,0.009006,-0.089209,0.154290,0.450530,-0.464430,0.270310,0.100940,-1.380900
1027,3.0,0,0,0,23.0,2,2,-0.517420,0.405430,-0.351090,...,-0.766800,-0.285950,1.320800,-0.169750,1.765100,-0.466960,-0.716560,0.291030,0.495080,-1.130600
1028,4.0,0,0,0,16.0,1,2,0.442194,-0.481770,-0.675675,...,-0.020160,0.184355,0.689670,-0.249810,1.255900,0.003750,-0.508270,0.085282,0.212090,-0.145425


In [4]:
df0 = df[df['location'] == 0]
df1 = df[df['location'] == 1]
df2 = df[df['location'] == 2]
df3 = df[df['location'] == 3]
df4 = df[df['location'] == 4]
df5 = df[df['location'] == 5]

df= df.drop('location', axis = 1)
df0= df0.drop('location', axis = 1)
df1= df1.drop('location', axis = 1)
df2= df2.drop('location', axis = 1)
df3= df3.drop('location', axis = 1)
df4= df4.drop('location', axis = 1)
df5= df5.drop('location', axis = 1)

In [22]:
# Perform train-test split for all dataframes 
Y_col = 'stars'
X_cols0 = df0.loc[:, df0.columns != Y_col].columns
X_cols1 = df1.loc[:, df1.columns != Y_col].columns
X_cols2 = df2.loc[:, df2.columns != Y_col].columns
X_cols3 = df3.loc[:, df3.columns != Y_col].columns
X_cols4 = df4.loc[:, df4.columns != Y_col].columns
X_cols5 = df5.loc[:, df5.columns != Y_col].columns

X_train0, X_test0, y_train0, y_test0 = train_test_split(df0[X_cols0], df0[Y_col],test_size=0.1, random_state=42)
X_train1, X_test1, y_train1, y_test1 = train_test_split(df1[X_cols1], df1[Y_col],test_size=0.1, random_state=42)
X_train2, X_test2, y_train2, y_test2 = train_test_split(df2[X_cols2], df2[Y_col],test_size=0.1, random_state=42)
X_train3, X_test3, y_train3, y_test3 = train_test_split(df3[X_cols3], df3[Y_col],test_size=0.1, random_state=42)
X_train4, X_test4, y_train4, y_test4 = train_test_split(df4[X_cols4], df4[Y_col],test_size=0.1, random_state=42)
X_train5, X_test5, y_train5, y_test5 = train_test_split(df5[X_cols5], df5[Y_col],test_size=0.1, random_state=42)


y_train0, y_test0 = np.array(list(y_train0)), np.array(list(y_test0))
y_train1, y_test1 = np.array(list(y_train1)), np.array(list(y_test1))
y_train2, y_test2 = np.array(list(y_train2)), np.array(list(y_test2))
y_train3, y_test3 = np.array(list(y_train3)), np.array(list(y_test3))
y_train4, y_test4 = np.array(list(y_train4)), np.array(list(y_test4))
y_train5, y_test5 = np.array(list(y_train5)), np.array(list(y_test5)) 

sc = StandardScaler()
X_train0, X_test0 = sc.fit_transform(X_train0), sc.transform(X_test0)
X_train1, X_test1 = sc.fit_transform(X_train1), sc.transform(X_test1)
X_train2, X_test2 = sc.fit_transform(X_train2), sc.transform(X_test2)
X_train3, X_test3 = sc.fit_transform(X_train3), sc.transform(X_test3)
X_train4, X_test4 = sc.fit_transform(X_train4), sc.transform(X_test4)
X_train5, X_test5 = sc.fit_transform(X_train5), sc.transform(X_test5)


X_train0, X_test0, y_train0, y_test0 = torch.from_numpy(X_train0).to(torch.float32), torch.from_numpy(X_test0).to(torch.float32), torch.from_numpy(y_train0).to(torch.float32), torch.from_numpy(y_test0).to(torch.float32)
X_train1, X_test1, y_train1, y_test1 = torch.from_numpy(X_train1).to(torch.float32), torch.from_numpy(X_test1).to(torch.float32), torch.from_numpy(y_train1).to(torch.float32), torch.from_numpy(y_test1).to(torch.float32)
X_train2, X_test2, y_train2, y_test2 = torch.from_numpy(X_train2).to(torch.float32), torch.from_numpy(X_test2).to(torch.float32), torch.from_numpy(y_train2).to(torch.float32), torch.from_numpy(y_test2).to(torch.float32)
X_train3, X_test3, y_train3, y_test3 = torch.from_numpy(X_train3).to(torch.float32), torch.from_numpy(X_test3).to(torch.float32), torch.from_numpy(y_train3).to(torch.float32), torch.from_numpy(y_test3).to(torch.float32)
X_train4, X_test4, y_train4, y_test4 = torch.from_numpy(X_train4).to(torch.float32), torch.from_numpy(X_test4).to(torch.float32), torch.from_numpy(y_train4).to(torch.float32), torch.from_numpy(y_test4).to(torch.float32)
X_train5, X_test5, y_train5, y_test5 = torch.from_numpy(X_train5).to(torch.float32), torch.from_numpy(X_test5).to(torch.float32), torch.from_numpy(y_train5).to(torch.float32), torch.from_numpy(y_test5).to(torch.float32)

In [27]:
class NeuralNetwork(nn.Module):
    def __init__(self, in_features, out_features, hidden_features, second_hidden_features):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(in_features, hidden_features),
            nn.ReLU(),
            nn.Linear(hidden_features, second_hidden_features),
            # # nn.Conv2d(hidden_features, hidden_features, 2),
            nn.ReLU(),
            nn.Linear(second_hidden_features, out_features)
        )
        # self.object = StandardScaler()
        # self.fc1 = nn.Linear(second_hidden_features, out_features)

    def forward(self, x):
        # x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        
        # logits = object.fit_transform(logits.detach().numpy()) 
        # logits = torch.from_numpy(logits).to(torch.float32)
        # logits = self.fc1(logits)
        return logits

In [28]:
# def define_model(in_features_size, in_channels_conv):
#   in_features = in_features_size
#   out_features = 1
#   hidden_features = 20
#   second_hidden_features = 5
#   learning_rate = 0.001
#   loss_function = nn.MSELoss() # Mean square error, useful in regression problems
#   model = NeuralNetwork(in_features, out_features, hidden_features, second_hidden_features, in_channels_conv)
#   optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#   return model, optimizer, loss_function

def define_model(in_features_size):
  in_features = in_features_size
  out_features = 1
  hidden_features = 20
  second_hidden_features = 5
  learning_rate = 0.001
  loss_function = nn.MSELoss() # Mean square error, useful in regression problems
  model = NeuralNetwork(in_features, out_features, hidden_features, second_hidden_features)
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  return model, optimizer, loss_function

In [29]:
def train_model(X_train, y_train, X_test, y_test):
  losses = {}
  accuracies = {}

  num_epochs = 1000
  # run with different amounts of data
  batch_split = [.2, .5, .8]
  for test_size in batch_split:
    batch_X_train, batch_X_test, batch_y_train, batch_y_test = train_test_split(
        X_train, y_train,test_size=test_size, random_state=42)
    print(batch_X_train.shape)
    model, optimizer, loss_function = define_model(len(batch_X_train[0]))
    # model, optimizer, loss_function = define_model(len(batch_X_train[0]), len(batch_X_train))
    train_losses = []
    test_losses = []
    train_accuracies = []
    test_accuracies = []

    for epoch in range(num_epochs):
      pred_y = model(batch_X_train).flatten()
      loss = loss_function(pred_y, batch_y_train)
      train_losses.append(loss.item())

      model.zero_grad()
      loss.backward()

      optimizer.step()

      # calc accuracy
      total = 0
      correct = 0
      total += batch_y_train.size(0)
      correct += np.sum(torch.squeeze(pred_y).round().detach().numpy() == batch_y_train.detach().numpy())
      accuracy = 100 * correct/total

      train_accuracies.append(accuracy)

      with torch.no_grad(): # do not update parameters

        # Calculating the loss and accuracy for the test dataset 
        correct_test = 0
        total_test = 0
        outputs_test = model(X_test).flatten()
        loss_test = loss_function(outputs_test, y_test)
        
        predicted_test = outputs_test.round().detach().numpy()
        total_test += y_test.size(0)
        correct_test += np.sum(predicted_test == y_test.detach().numpy())
        accuracy_test = 100 * correct_test/total_test

        test_losses.append(loss_test.item()) # append test loss
        test_accuracies.append(accuracy_test) # append test accuracy
        

      # print(f"\nEpoch: {epoch}. \nTest - Loss: {loss_test.item()}. Accuracy: {accuracy_test}")
      print(f"Epoch: {epoch}. Train - Loss: {loss.item()}. Accuracy: {accuracy}  Test -  Loss: {loss_test.item()}. Accuracy: {accuracy_test}\n")

    accuracies[test_size] = [train_accuracies, test_accuracies]
    losses[test_size] = [train_losses, test_losses]
  return accuracies, losses, model

In [30]:
accuracy0, loss0, model0 = train_model(X_train0, y_train0, X_test0, y_test0)
accuracy1, loss1, model1 = train_model(X_train1, y_train1, X_test1, y_test1)
accuracy2, loss2, model2 = train_model(X_train2, y_train2, X_test2, y_test2)
accuracy3, loss3, model3 = train_model(X_train3, y_train3, X_test3, y_test3)
accuracy4, loss4, model4 = train_model(X_train4, y_train4, X_test4, y_test4)
accuracy5, loss5, model5 = train_model(X_train5, y_train5, X_test5, y_test5)
accuracies = [accuracy0, accuracy1, accuracy2, accuracy3, accuracy4, accuracy5]
losses = [loss0, loss1, loss2, loss3, loss4, loss5]

Streaming output truncated to the last 5000 lines.
Epoch: 501. Train - Loss: 0.1074506938457489. Accuracy: 47.36842105263158  Test -  Loss: 0.6096192002296448. Accuracy: 31.57894736842105

Epoch: 502. Train - Loss: 0.10713692009449005. Accuracy: 47.36842105263158  Test -  Loss: 0.6092923283576965. Accuracy: 31.57894736842105

Epoch: 503. Train - Loss: 0.10683389008045197. Accuracy: 47.36842105263158  Test -  Loss: 0.6089988946914673. Accuracy: 31.57894736842105

Epoch: 504. Train - Loss: 0.10652794688940048. Accuracy: 47.36842105263158  Test -  Loss: 0.6087988018989563. Accuracy: 31.57894736842105

Epoch: 505. Train - Loss: 0.10622229427099228. Accuracy: 47.36842105263158  Test -  Loss: 0.608607828617096. Accuracy: 31.57894736842105

Epoch: 506. Train - Loss: 0.10592658817768097. Accuracy: 47.36842105263158  Test -  Loss: 0.6084979772567749. Accuracy: 31.57894736842105

Epoch: 507. Train - Loss: 0.10562794655561447. Accuracy: 47.36842105263158  Test -  Loss: 0.6083763241767883. Accurac

In [32]:
# for all samples in vaughan df:
# find model output of model0
# find actual star value of that sample
# add the toroto:vaughan ratio to the ratios array
# at the end, find the average ratio. this represents moving a restaurant from vaughan to toroto 
def get_coefficient(model, X_train, y_train):
  # coefficient = moving a restaurant from train_city to model_city
  ratios = [0] # same features in model_city : same features in train_city
  for ix, (x, y) in enumerate(zip(X_train, y_train)):
    model_output = model(x)[0].item()
    if y != 0:
      ratios.append(model_output / y.item())
  return sum(ratios) / len(ratios)

In [45]:
models = [model0, model1, model2, model3, model4, model5]
X_trains = [X_train0, X_train1, X_train2, X_train3, X_train4, X_train5]
y_trains = [y_train0, y_train1, y_train2, y_train3, y_train4, y_train5]
coeffs = [ [ 0 for _ in range(6) ] for _ in range(6) ]

In [46]:
for i in range(len(models)):
  for j in range(len(X_trains)):
    if i == j:
      coeffs[i][j] = 1
    else:
      coeffs[i][j] = get_coefficient(models[i], X_trains[j], y_trains[j])
coeffs = pd.DataFrame(coeffs, columns =["From Toronto", "From Mississauga", "From Vaughan", "From Richmond Hill", "From Brampton", "From Markham"]) 
coeffs = coeffs.rename(index={0:"To Toronto", 1: "To Mississauga", 2: "To Vaughan", 3: "To Richmond Hill", 4: "To Brampton", 5: "To Markham"})
coeffs = coeffs.T
coeffs

,To Toronto,To Mississauga,To Vaughan,To Richmond Hill,To Brampton,To Markham
From Toronto,1.000000,0.794577,0.831074,0.618634,0.692919,0.717700
From Mississauga,0.849390,1.000000,0.860742,0.634656,0.734907,0.754298
From Vaughan,0.868970,0.880705,1.000000,0.690579,0.742406,0.808719
From Richmond Hill,0.864531,0.732666,0.863656,1.000000,0.704259,0.793879
From Brampton,0.875172,0.933174,0.924535,0.657119,1.000000,0.794501
From Markham,0.877546,0.763446,0.906459,0.740752,0.714988,1.000000
